## 4.4 ListOfLists

Wykorzystamy `Jetpack Navigation` wraz z `RecyclerView` aby stworzyć prostą aplikację w modelu **master-detail**.

### **Fragmenty**

W aplikacji będziemy posiadać dwa fragmenty hostowane przez aktywność. Oba fragmenty będą zawierać `RecyclerView` - na jednym będzie znajdowała się lista liter, nac drugim lista słów na daną literę. Rozpocznijmy od dodania dwóch fragmentów.

In [ ]:
class LetterFragment : Fragment() {

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        return inflater.inflate(R.layout.fragment_letter, container, false)
    }
}

```xml
<FrameLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".LetterFragment">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/letterRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:clipToPadding="false"
        android:padding="16dp" />

</FrameLayout>
```

In [ ]:
class WordFragment : Fragment() {

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        return inflater.inflate(R.layout.fragment_word, container, false)
    }
}

```xml
<FrameLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    tools:context=".WordFragment">

    <androidx.recyclerview.widget.RecyclerView
        android:id="@+id/wordRecyclerView"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        android:clipToPadding="false"
        android:padding="16dp" />

</FrameLayout>
```

### **Adaptery**

W kolejnym kroku napiszemy adaptery dla liter i słów. Rozpocznijmy od layoutu który będą wykorzystywały oba adaptery (`item_view.xml`)

```xml
<?xml version="1.0" encoding="utf-8"?>
<Button xmlns:android="http://schemas.android.com/apk/res/android"
    android:id="@+id/button_item"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_margin="4dp"
    android:padding="8dp" />
```

Przejdźmy do `LetterAdapter`

In [ ]:
class LetterAdapter : RecyclerView.Adapter<LetterAdapter.LetterViewHolder>(){
    private val list = ('A').rangeTo('Z').toList()

    class LetterViewHolder(private val view: View) : RecyclerView.ViewHolder(view) {
        val button: Button = view.findViewById(R.id.button_item)
    }

    override fun getItemCount(): Int = list.size

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): LetterViewHolder {
        return LetterViewHolder(LayoutInflater
            .from(parent.context)
            .inflate(R.layout.item_view, parent, false))
    }

    override fun onBindViewHolder(holder: LetterViewHolder, position: Int) {
        val item = list[position]
        holder.button.text = item.toString()
    }
}

`WordAdapter` będzie zawierał listę słów zaczynających się na literę, którą użytkownik wybierze na ekranie `FragmentLetter` - lista wszystkich śłów znajduje się w pliku `arrays.xml`

In [ ]:
class WordAdapter(private val letter: String, context: Context) :
    RecyclerView.Adapter<WordAdapter.WordViewHolder>() {}

W konstruktorze głównym `WordAdapter` przyjmuje literę, który będzie wykorzystywany do przefiltrowania listy wszystkich słów. Wczytywanie oraz filtrowanie wykonujemy w bloku `init`

In [ ]:
class WordAdapter(private val letterId: String, context: Context) :
    RecyclerView.Adapter<WordAdapter.WordViewHolder>() {

    private val words: List<String>

    init {
        words = context.resources.getStringArray(R.array.words).toList()
            .filter { it.startsWith(letterId, ignoreCase = true) }
    }

    class WordViewHolder(private val view: View) : RecyclerView.ViewHolder(view) {
        val button: Button = view.findViewById(R.id.button_item)
    }

    override fun getItemCount(): Int = words.size

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): WordViewHolder {
        return WordViewHolder(LayoutInflater
            .from(parent.context)
            .inflate(R.layout.item_view, parent, false))
    }

    override fun onBindViewHolder(holder: WordViewHolder, position: Int) {
        val item = words[position]
        holder.button.text = item
    }
}

Dodajmy `RecyclerView` do `LetterFragment` oraz połączmy go z `LetterAdapter` -wykonujemy to w metodzie `onViewCreated`

In [ ]:
class LetterFragment : Fragment() {

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View? {
        return inflater.inflate(R.layout.fragment_letter, container, false)
    }

    override fun onViewCreated(view: View, savedInstanceState: Bundle?) {
        super.onViewCreated(view, savedInstanceState)
        val recyclerView: RecyclerView = view.findViewById(R.id.letterRecyclerView)
        recyclerView.layoutManager = LinearLayoutManager(context)
        recyclerView.adapter = LetterAdapter()
    }
}

### **Nawigacja**